# DATA PRE TREATMENT

Let's start by importing all our **Libraries**

In [47]:
# IMPORT
## Actually uses
import sys
import os
import pandas as pd
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import validation_curve
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC

## Actually not uses 
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from mpl_toolkits import mplot3d
from itertools import combinations
from itertools import chain
%matplotlib inline

## Importing the Dataset

In [25]:
# PATHS
DATA_PATH = os.path.join("./data")
DATA_NAME_CSV = "CyberTroll.csv"

Extraction of our data and transformation into DataFrame with **pandas**

In [26]:
#Transforming the .csv file into a DataFrame
def load_data(data_path=DATA_PATH,data_name_csv=DATA_NAME_CSV):
        # Path to file mushrooms.csv
        csv_path = os.path.join(data_path,data_name_csv)
        # Object DATA_FRAME
        return pd.read_csv(csv_path)

# execution
data_frame = load_data()

Let's look at what this data looks like

In [27]:
data_frame.head()

,content,annotation/notes,annotation/label/0,extras,metadata/first_done_at,metadata/last_updated_at,metadata/sec_taken,metadata/last_updated_by,metadata/status,metadata/evaluation
0,Get fucking real dude.,NaN,1,NaN,1527503426000,1527503426000,0,jI67aE5hwwdh6l16bcfFVnpyREd2,done,NONE
1,She is as dirty as they come and that crook R...,NaN,1,NaN,1527503426000,1527503426000,0,jI67aE5hwwdh6l16bcfFVnpyREd2,done,NONE
2,why did you fuck it up. I could do it all day ...,NaN,1,NaN,1527503426000,1527503426000,0,jI67aE5hwwdh6l16bcfFVnpyREd2,done,NONE
3,Dude they dont finish enclosing the fucking sh...,NaN,1,NaN,1527503426000,1527503426000,0,jI67aE5hwwdh6l16bcfFVnpyREd2,done,NONE
4,WTF are you talking about Men? No men thats no...,NaN,1,NaN,1527503426000,1527503426000,0,jI67aE5hwwdh6l16bcfFVnpyREd2,done,NONE


Quick check of the quantity of samples and the diversity of cases

In [28]:
data_frame.describe() 

,annotation/notes,annotation/label/0,extras,metadata/first_done_at,metadata/last_updated_at,metadata/sec_taken
count,0.0,20001.000000,0.0,2.000100e+04,2.000100e+04,20001.0
mean,NaN,0.391080,NaN,1.527503e+12,1.527503e+12,0.0
std,NaN,0.488005,NaN,2.717817e+04,2.717817e+04,0.0
min,NaN,0.000000,NaN,1.527503e+12,1.527503e+12,0.0
25%,NaN,0.000000,NaN,1.527503e+12,1.527503e+12,0.0
50%,NaN,0.000000,NaN,1.527503e+12,1.527503e+12,0.0
75%,NaN,1.000000,NaN,1.527503e+12,1.527503e+12,0.0
max,NaN,1.000000,NaN,1.527504e+12,1.527504e+12,0.0


Check if there is a null value and all the indexes we have

In [29]:
data_frame.isnull().sum()

content                         0
annotation/notes            20001
annotation/label/0              0
extras                      20001
metadata/first_done_at          0
metadata/last_updated_at        0
metadata/sec_taken              0
metadata/last_updated_by        0
metadata/status                 0
metadata/evaluation             0
dtype: int64

**annotation/notes**,**extras** is composed of null values so its implication in a classification algorithm is not relevant.

In [30]:
data_frame = data_frame.drop(['annotation/notes','extras'],axis=1)

Let's look at all the unique choices we have in all indexes, if there is an index that always has the same value we can remove it.

In [31]:
for index in data_frame.columns:
        print(str(index) + " : " + str(data_frame[index].unique()))

content : ['Get fucking real dude.'
 "She is as dirty as they come  and that crook Rengel  the Dems are so fucking corrupt it's a joke. Make Republicans look like  ..."
 "why did you fuck it up. I could do it all day too. Let's do it when you have an hour. Ping me later to sched writing a book here."
 ... 'hahahahaha >:) im evil mwahahahahahahahahaha'
 'What&;s something unique about Ohio? :)'
 'Who is the biggest gossiper you know?']
annotation/label/0 : [1 0]
metadata/first_done_at : [1527503426000 1527503427000 1527503428000 1527503429000 1527503430000
 1527503431000 1527503432000 1527503433000 1527503434000 1527503435000
 1527503436000 1527503437000 1527503438000 1527503439000 1527503440000
 1527503441000 1527503442000 1527503443000 1527503444000 1527503445000
 1527503446000 1527503447000 1527503448000 1527503449000 1527503450000
 1527503451000 1527503452000 1527503453000 1527503454000 1527503455000
 1527503456000 1527503457000 1527503458000 1527503459000 1527503460000
 15275034610

**metadata/sec_taken**,**metadata/last_updated_by**,**metadata/status**,**metadata/evaluation** is composed of a single variable so its implication in a classification algorithm is not relevant.

In [32]:
data_frame = data_frame.drop(['metadata/sec_taken','metadata/last_updated_by','metadata/status','metadata/evaluation'],axis=1)

**metadata/last_updated_at**,**metadata/first_done_at** is composed of timestamp so it's implication in not relevant.

In [33]:
data_frame = data_frame.drop(['metadata/last_updated_at','metadata/first_done_at'],axis=1)

In [34]:
data_frame.head()

,content,annotation/label/0
0,Get fucking real dude.,1
1,She is as dirty as they come and that crook R...,1
2,why did you fuck it up. I could do it all day ...,1
3,Dude they dont finish enclosing the fucking sh...,1
4,WTF are you talking about Men? No men thats no...,1


Rename columns for easier use:

In [35]:
data_frame = data_frame.rename(columns={"content" : "sentence","annotation/label/0" :"label"})

## Text Preprocessing

Text may contain numbers, special characters, and unwanted spaces. We will remove all the special characters, numbers, and unwanted spaces from our text.

In [36]:
# Lemmatizer 
stemmer = WordNetLemmatizer()
# Function to apply on our sentence
def converterSentence(text):
    # Remove all the special characters
    text = re.sub(r'\W', ' ', text)
    # remove all single characters
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    # Remove single characters from the start
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    # Substituting multiple spaces with single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    # Converting to Lowercase
    text = text.lower()
    # Lemmatization
    text = text.split()
    text = [stemmer.lemmatize(word) for word in text]
    text = ' '.join(text)
    return text

In [37]:
data_frame['sentence']= data_frame.sentence.apply(converterSentence)
data_frame.sentence

0                                    get fucking real dude
1        she is a dirty a they come and that crook reng...
2        why did you fuck it up could do it all day too...
3        dude they dont finish enclosing the fucking sh...
4        wtf are you talking about men no men thats not...
5        ill save you the trouble sister here come big ...
6        im dead serious real athlete never cheat don e...
7        go absolutely insane hate to be the bearer of ...
8        lmao im watching the same thing ahaha the gay ...
9        lol no he said what do you call jail cell to g...
10       truth on both count that guy is an as and thei...
11                                        shakespeare nerd
12                               you are such fucking dork
13                                       heh fuck em where
14                         damn it totally forgot that one
15                    wow damn would have been pissed that
16               nigga geigh lmao fuck yo final beeeeeit

# Analysing Data

Different approaches exist to convert text into the corresponding numerical form. But we are going to use Bag of Words Model because it is the most efficient.

**Default Vectorizer & Default model to have a first overview of the result** :

In [38]:
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(data_frame.sentence)
Y = data_frame.label

The bag of words approach works fine for converting text to numbers. However, it has one drawback. It assigns a score to a word based on its occurrence in a particular document. It doesn't take into account the fact that the word might also be having a high frequency of occurrence in other documents as well. TFIDF resolves this issue by multiplying the term frequency of a word by the inverse document frequency. 

In [39]:
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X)

In [40]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [41]:
# Simple Random Forest Classifier
classifier = RandomForestClassifier(n_estimators=20, random_state=0)
classifier.fit(x_train, y_train) 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [19]:
y_pred_rdtree = classifier.predict(x_test)
print(classification_report(y_test,y_pred_rdtree))

             precision    recall  f1-score   support

          0       0.96      0.91      0.93      2429
          1       0.87      0.93      0.90      1572

avg / total       0.92      0.92      0.92      4001



In [20]:
confusion_rdtree = confusion_matrix(y_test, y_pred_rdtree)
print("We obtain the following confusion matrix : \n", confusion_rdtree)

We obtain the following confusion matrix : 
 [[2216  213]
 [ 103 1469]]


We're using RandomizedSearchCV in order to optimize by cross-validated search the hyperparameters.

In [21]:
#Finding the best estimator : the best hyperparameters

# Number of trees in random forest
n_estimators = [100, 300, 500, 800, 1200]
# Maximum number of levels in tree
max_depth = [5, 8, 15, 25, 30]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10] 

random_grid = dict(n_estimators = n_estimators, 
              max_depth = max_depth,  
              min_samples_split = min_samples_split, 
              min_samples_leaf = min_samples_leaf)

rf_random = RandomizedSearchCV(estimator = RandomForestClassifier(), 
                               param_distributions = random_grid, 
                               n_iter = 100, 
                               cv = 3, 
                               verbose=2, 
                               random_state=42, 
                               n_jobs = -1)
rf_random.fit(x_train, y_train)
rf_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 14.7min finished


{'n_estimators': 800,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_depth': 30}

We are going to  plot the validation curve to find the best value for the n_estimators

In [18]:
# train_scores, test_scores = validation_curve(
#                                 RandomForestClassifier(),
#                                 X = x_train, y = y_train, 
#                                 param_name = 'n_estimators', 
#                                 param_range = [100, 300, 500, 750, 800, 1200], cv = 3)

# param_range = np.logspace(-6, -1, 5)
# train_scores_mean = np.mean(train_scores, axis=1)
# test_scores_mean = np.mean(test_scores, axis=1)

# plt.title("Validation Curve for Random Forest")
# plt.xlabel("Values of Hyperparameter n_estimators")
# plt.ylabel("Accuracy Score")

# plt.plot(param_range, train_scores_mean, label="Training scores")
# plt.plot(param_range, test_scores_mean, label="Cross-Validation Score")

# plt.show()

Attempt to find parameters for the vectorizer which could impact the model. But the sentences being too small the min_df and max_df parameters are useless here. We still just tried to vary the number of features but that did not affect the accuracy.

In [23]:
LIST_MIN_DEF= [x/100 for x in range(1,50,5)]
LIST_MAX_DEF= [x for x in range(90,100)]
LIST_MAX_DEF.reverse()
LIST_MAX_FEATURES = [x for x in range(100,2000,50)]
def customVectorizer(listMinDef=LIST_MIN_DEF,listMaxDef=LIST_MAX_DEF,listMaxFeatures=LIST_MAX_FEATURES):
    listVectorizer = []
    for maxFeatures in listMaxFeatures:
        listVectorizer.append(CountVectorizer(stop_words='english',max_features=maxFeatures))  
    return listVectorizer


You can find the result of this function in a file located in the data folder, just for information there is just marked the same for each variation.

In [24]:
# Vary vectorizer parameters to see if it impacts the default Random Forest model
def ImpactOfVectorizer():
    fichier = open("data/dataRFCDefaultCustomVectorizer.txt", "a")
    for vecto in customVectorizer():
        vectorizer = vecto
        X = vectorizer.fit_transform(data_frame.sentence)
        Y = data_frame.label
        tfidfconverter = TfidfTransformer()
        X = tfidfconverter.fit_transform(X)
        x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
        # Simple Random Forest Classifier
        classifier = RandomForestClassifier(n_estimators=10, random_state=0)
        classifier.fit(x_train, y_train) 
        crboost = classification_report(y_test,y_pred,digits=10,output_dict=True)
        fichier.write(str(crboost['0']['precision'])+" " +str(crboost['1']['precision'])+"\n")
    fichier.close()
# The famous line that saves you 5 s               
# ImpactOfVectorizer()

Result :
<br/> 0.9555843035791289 0.8733650416171225
<br/> 0.9555843035791289 0.8733650416171225
<br/> 0.9555843035791289 0.8733650416171225
<br/> 0.9555843035791289 0.8733650416171225
<br/> ...

"The answer to the question “What machine learning model should I use?” is always “It depends.” Even the most experienced data scientists can’t tell which algorithm will perform best before experimenting them."

In order to find the optimum model, we tried to make as many models as possible to know which would be the best. Here are the 6 models that were made.

MLPClassifier, DecisionTreeClassifier, RandomForestClassifier, KNeighborsClassifier, LogisticRegression and finally SVM (Support Vector Machine).

We will use these trains and tests for all our models.

In [53]:
# Vectorizer
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(data_frame.sentence)
Y = data_frame.label
# TFID
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X)
# SPLIT
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

## MLPClassifier

This model aims to optimize the log-loss function using LBGFGS or stochastic gradient descent.

We will start with the default neural network (45 min)

In [24]:
modelMLP = MLPClassifier()
modelMLP.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

Using the trained network to predict

In [25]:
y_pred = modelMLP.predict(x_test)
modelMLP.score(x_test, y_pred)

1.0

In [26]:
conf_matrix = confusion_matrix(y_test,y_pred)
conf_matrix

array([[2892,  770],
       [ 202, 2137]], dtype=int64)

In [27]:
cr = classification_report(y_test,y_pred)
print(cr)

             precision    recall  f1-score   support

          0       0.93      0.79      0.86      3662
          1       0.74      0.91      0.81      2339

avg / total       0.86      0.84      0.84      6001



We will now try to find more optimal parameters, with the magic GridSearchCV multi-threads.

**A EXECUTER A PARTIR D'ICI**

In [ ]:
modelMLP = MLPClassifier()
parameter_space = {'activation': ['logistic','tanh', 'relu'],
                  'hidden_layer_sizes':[i for i in range(1,220,20)],
                  'alpha':[0.0001,0.001,0.01,0.1,1],
                  'max_iter':[50, 100, 150, 200]}
gridSearch = GridSearchCV(modelMLP, parameter_space, n_jobs=-1, cv=3)
gridSearch.fit(x_train,y_train)

Let's the best results:

In [ ]:
# Best parameters set
print('Best parameters found:\n', gridSearch.best_params_)

In [ ]:
# All results
means = gridSearch.cv_results_['mean_test_score']
stds = gridSearch.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, gridSearch.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

**STOP EXECUTION ICI**

## LogisticRegression

This classification is used in statistics to model the probability of a certain class.

We will start with the default one.

In [45]:
modelLR = LogisticRegression()
modelLR.fit(x_train,y_train)

/home/tchouka/ml/env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
y_pred = modelLR.predict(x_test)
modelLR.score(x_test, y_pred)

1.0

In [47]:
conf_matrix = confusion_matrix(y_test,y_pred)
conf_matrix

array([[3204,  458],
       [ 982, 1357]])

In [48]:
cr = classification_report(y_test,y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.77      0.87      0.82      3662
           1       0.75      0.58      0.65      2339

    accuracy                           0.76      6001
   macro avg       0.76      0.73      0.73      6001
weighted avg       0.76      0.76      0.75      6001



We will now try to add hyperparameter to get a better accuracy.

For this one, 4 hyperparameters : C, penality, max_iter and tol.(10 min)

In [52]:
modelLR = LogisticRegression()
parameter_space = {'penalty': ['l1','l2'],
                  'C':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                  'max_iter':[50, 100, 150, 200],
                  'tol':[0.00001,0.0001, 0.001, 0.01, 0.1, 1, 10]}
gridSearchLR = GridSearchCV(modelLR, parameter_space, n_jobs=-1, cv=3)
gridSearchLR.fit(x_train,y_train)

/home/tchouka/ml/env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/tchouka/ml/env/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                         'max_iter': [50, 100, 150, 200],
                         'penalty': ['l1', 'l2'],
                         'tol': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             sco

Let's see the best results:

In [55]:
# Best parameter set
print('Best parameters found:\n', gridSearchLR.best_params_)

Best parameters found:
 {'C': 10, 'max_iter': 100, 'penalty': 'l1', 'tol': 1e-05}


In [56]:
# All results
means = gridSearchLR.cv_results_['mean_test_score']
stds = gridSearchLR.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, gridSearchLR.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

0.608 (+/-0.000) for {'C': 0.0001, 'max_iter': 50, 'penalty': 'l1', 'tol': 1e-05}
0.608 (+/-0.000) for {'C': 0.0001, 'max_iter': 50, 'penalty': 'l1', 'tol': 0.0001}
0.608 (+/-0.000) for {'C': 0.0001, 'max_iter': 50, 'penalty': 'l1', 'tol': 0.001}
0.608 (+/-0.000) for {'C': 0.0001, 'max_iter': 50, 'penalty': 'l1', 'tol': 0.01}
0.608 (+/-0.000) for {'C': 0.0001, 'max_iter': 50, 'penalty': 'l1', 'tol': 0.1}
0.608 (+/-0.000) for {'C': 0.0001, 'max_iter': 50, 'penalty': 'l1', 'tol': 1}
0.608 (+/-0.000) for {'C': 0.0001, 'max_iter': 50, 'penalty': 'l1', 'tol': 10}
0.608 (+/-0.000) for {'C': 0.0001, 'max_iter': 50, 'penalty': 'l2', 'tol': 1e-05}
0.608 (+/-0.000) for {'C': 0.0001, 'max_iter': 50, 'penalty': 'l2', 'tol': 0.0001}
0.608 (+/-0.000) for {'C': 0.0001, 'max_iter': 50, 'penalty': 'l2', 'tol': 0.001}
0.608 (+/-0.000) for {'C': 0.0001, 'max_iter': 50, 'penalty': 'l2', 'tol': 0.01}
0.608 (+/-0.000) for {'C': 0.0001, 'max_iter': 50, 'penalty': 'l2', 'tol': 0.1}
0.608 (+/-0.000) for {'C': 

Let's take a closer look at the results of the best parameters

In [61]:
modelLR = LogisticRegression(C=10,max_iter=100,penalty='l1',tol=1e-05)
modelLR.fit(x_train,y_train)
y_pred = modelLR.predict(x_test)
conf_matrix = confusion_matrix(y_test,y_pred)
conf_matrix

array([[3165,  497],
       [ 306, 2033]])

In [62]:
cr = classification_report(y_test,y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.91      0.86      0.89      3662
           1       0.80      0.87      0.84      2339

    accuracy                           0.87      6001
   macro avg       0.86      0.87      0.86      6001
weighted avg       0.87      0.87      0.87      6001



Test de LogisticRegressionCV

In [54]:
clf = LogisticRegressionCV(cv=5, random_state=0).fit(x_train, y_train)
clf.predict(X[:2, :])

array([1, 1], dtype=int64)

In [55]:
clf.score(x_train, y_train)

0.966

## KneighborsClassifier

This classifier uses the k-nearest neighbors method.

Here we go for execution by default.(10h)

In [70]:
modelKNC = KNeighborsClassifier()
modelKNC.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [72]:
y_pred = modelKNC .predict(x_test)
modelKNC .score(x_test, y_pred)

1.0

In [73]:
conf_matrix = confusion_matrix(y_test,y_pred)
conf_matrix

array([[3453,  209],
       [1661,  678]])

In [74]:
cr = classification_report(y_test,y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.68      0.94      0.79      3662
           1       0.76      0.29      0.42      2339

    accuracy                           0.69      6001
   macro avg       0.72      0.62      0.60      6001
weighted avg       0.71      0.69      0.64      6001



Once again it's time to do magic and find the best hyperparameters. We will try to vary the algorithm parameter, n_neighbors and weights.

**EXECUTION D'ICI**

In [ ]:
modelKN = KNeighborsClassifier()
parameter_space = {'algorithm': ['ball_tree','auto', 'kd_tree', 'brute'],
                  'n_neighbors':[i for i in range(30,100,10)],
                  'weights':['distance','uniform']}
gridSearchKN = GridSearchCV(modelKN, parameter_space, n_jobs=-1, cv=3)
gridSearchKN.fit(x_train,y_train)

Let's see the best result:

In [23]:
# Best parameter set
print('Best parameters found:\n', gridSearchKN.best_params_)

NameError: name 'gridSearchKN' is not defined

In [25]:
# All results
means = gridSearchKN.cv_results_['mean_test_score']
stds = gridSearchKN.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, gridSearchKN.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

NameError: name 'gridSearchKN' is not defined

Let's take a closer look at the results of the best parameters

In [26]:
modelKN = KNeighborsClassifier(A REMPLIR)
modelKN.fit(x_train,y_train)
y_pred = modelKN.predict(x_test)
conf_matrix = confusion_matrix(y_test,y_pred)
conf_matrix

SyntaxError: invalid syntax (<ipython-input-26-2447c423a874>, line 1)

In [27]:
cr = classification_report(y_test,y_pred)
print(cr)

ValueError: Found input variables with inconsistent numbers of samples: [6001, 4001]

**FIN EXECUTION**

# Linear Support Vector Machine Classifier

The SVM algorithm finds the right hyperplans that differentiate classes.

In [42]:
clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
conf_matrix_clf = confusion_matrix(y_test,y_pred)
conf_matrix_clf

array([[2097,  332],
       [ 204, 1368]], dtype=int64)

In [43]:
clf.score(x_test, y_pred)

1.0

In [44]:
class_clf = classification_report(y_test, y_pred)
print(class_clf)

             precision    recall  f1-score   support

          0       0.91      0.86      0.89      2429
          1       0.80      0.87      0.84      1572

avg / total       0.87      0.87      0.87      4001

